In [11]:
# packages

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import pandas as pd
import os
from itertools import combinations
import h5py

import sys
sys.path.append("../src")

from analysis import *
from inference import *

In [12]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

In [13]:
datapath = "../experiment_outputs/growth_scale_0.1_10_sp_20_param_seed_50_init_cond_env_noise0.0"
log = h5py.File(f"{datapath}/data_generation_log.h5", "r")

print(f"n_species = {log.attrs['n_species']}")
print(f"avg_samp_dt = {log.attrs['avg_samp_dt']}")
print(f"env_noise = {log.attrs['env_noise']}")
print(f"meas_noise_list = {log.attrs['meas_noise_list']}")
print(f"n_params_seeds = {log.attrs['n_params_seeds']}")

n_species = [10]
avg_samp_dt = [3. 1.]
env_noise = 0.0
meas_noise_list = [0.]
n_params_seeds = 20


In [14]:
def get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt, filetype="dataset", ext="csv"):
    params_seeds = [i.split("param_seed")[1] for i in os.listdir(f"{datapath}/{n_sp}_sp")]

    datafiles = []

    for p in params_seeds:
        datafiles.append(f"{datapath}/{n_sp}_sp/param_seed{p}/meas_noise{meas_noise}/t_samp{avg_samp_dt}/{filetype}{n_sp}_sp{p}_env_noise{env_noise}.{ext}")
    return datafiles

In [15]:
print(f"Numbers of sampling points: {log.attrs['n_samples']}")
print(f"Average sampling intervals: {log.attrs['avg_samp_dt'].round(3)}")
print(f"Number of initial conditions: {log.attrs['n_init_cond']}")
print(f"Number of repetitions: {log.attrs['repetitions']}")
print(f"Environmental noise: {log.attrs['env_noise']}")
print(f"Amounts of measurement noise: {log.attrs['meas_noise_list']}")

env_noise = log.attrs['env_noise']

Numbers of sampling points: [11 31]
Average sampling intervals: [3. 1.]
Number of initial conditions: 50
Number of repetitions: 1
Environmental noise: 0.0
Amounts of measurement noise: [0.]


In [16]:
def calculate_es_score(true_aij, inferred_aij) -> float:
    """GRANT'S edited version to calculate ED score

    Calculate the ecological direction (EDₙ) score (n := number of species in ecosystem).

    Parameters
    ===============
    truth: ndarray(axis0=species_names, axis1=species_names), the ecosystem coefficient matrix used to generate data
    inferred: ndarray(axis0=species_names, axis1=species_names), the inferred ecosystem coefficient matrix
    Returns
    ===============
    ES_score: float
    """

    truth = pd.DataFrame(true_aij).copy()
    inferred = pd.DataFrame(inferred_aij).copy()

    # consider inferred coefficients
    mask = inferred != 0

    # compare sign: agreement when == -2 or +2, disagreement when 0
    nonzero_sign = np.sign(inferred)[mask] + np.sign(truth)[mask]
    corr_sign = (np.abs(nonzero_sign) == 2).sum().sum()
    opposite_sign = (np.abs(nonzero_sign) == 0).sum().sum()

    # count incorrect non-zero coefficients
    wrong_nz = (truth[mask] == 0).sum().sum()

    # combine
    unscaled_score = corr_sign - opposite_sign

    # scale by theoretical extrema
    truth_nz_counts = (truth != 0).sum().sum()
    truth_z_counts = len(truth.index) ** 2 - truth_nz_counts
    theoretical_min = -truth_nz_counts
    theoretical_max = truth_nz_counts

    ES_score = (unscaled_score - theoretical_min) / (theoretical_max - theoretical_min)

    return ES_score

In [17]:
import math

def n_comb(n, k):
    return math.factorial(n)/(math.factorial(n-k)*math.factorial(k))

# Infer and score

for n_sp in log.attrs["n_species"]:
    for avg_samp_dt in log.attrs["avg_samp_dt"]:
        for meas_noise in log.attrs["meas_noise_list"]:
            datafiles = get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt)
            metadatafiles = get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt, "metadata", "txt")

            for file_idx in range(len(datafiles)):
                datafile = datafiles[file_idx]
                metadatafile = metadatafiles[file_idx]
                metadict = get_meta(open(metadatafile, "r").read().split("\n"))
                
                df = pd.read_csv(datafile, index_col=0)
                
                param_columns = [f"r{i}" for i in range(1, n_sp+1)] + \
                [f"A{i},{j}" for i in range(1, n_sp+1) for j in range(1, n_sp+1)]
                cols = ["n_init_cond"] + list(df.columns[1:4]) + param_columns + ["MSPD", "CSR", "ES"]

                infer_out = pd.DataFrame(columns=cols)

                pd.options.mode.chained_assignment = None
                
                p = metadict["parameters"]
                r = p[:n_sp]
                A = p[n_sp:].reshape((n_sp,n_sp))

                # for i in tqdm(range(len(df.init_cond_idx.unique()))):
                for i in tqdm(range(30)):
                    if n_comb(len(df.dataset.unique()), i+1) < 10000:
                        combs = list(combinations(df.dataset.unique(), i+1))
                        np.random.shuffle(combs)
                        combs = combs[:100]
                    else:
                        combs = []
                        while len(combs) < 100:
                            comb = tuple(np.random.choice(df.dataset.unique(), i+1, replace=False))
                            if comb not in combs:
                                combs.append(comb)
                    combs = list(combinations(df.init_cond_idx.unique(), i+1))
                    np.random.shuffle(combs)
                    for comb in combs[:100]:
                        df_comb = df[df.init_cond_idx.isin(comb)]
                        r_est, A_est = fit_ridge_cv(df_comb)
                        p_est = np.concatenate((r_est, A_est.flatten()))
                        MSPD = ((p-p_est)**2).mean()
                        CSR = (np.sign(A_est)==np.sign(A)).mean()
                        ES = calculate_es_score(A, A_est)
                        infer_out.loc[len(infer_out)] = [i+1, comb, avg_samp_dt, meas_noise] + list(p_est) + [MSPD, CSR, ES]

                infer_out.to_csv(datafile.split('dataset')[0]+"/inference"+datafile.split("dataset")[1])

In [19]:
n_sp = 10
avg_samp_dt = 3.
env_noise = 0.
meas_noise = 0.

In [20]:
datafiles = get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt)
pd.read_csv(datafiles[0], index_col=0)

,dataset,init_cond_idx,t_samp_dist_idx,measurement_noise,replicate,time,dt,sp1,sp2,sp3,sp4,sp5,sp6,sp7,sp8,sp9,sp10
0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.021630,0.028249,0.013840,0.022642,0.008141,0.009654,0.005626,0.038545,0.004096,0.010321
1,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.059846,0.094365,0.041658,0.091716,0.022285,0.034693,0.025160,0.127579,0.048889,0.015993
2,0.0,0.0,0.0,0.0,0.0,6.0,3.0,0.120860,0.094792,0.085598,0.181751,0.043966,0.053917,0.072112,0.092637,0.182284,0.034928
3,0.0,0.0,0.0,0.0,0.0,9.0,3.0,0.190292,0.066873,0.137012,0.205234,0.064095,0.077086,0.111342,0.052580,0.201673,0.137759
4,0.0,0.0,0.0,0.0,0.0,12.0,3.0,0.230442,0.067420,0.171315,0.207259,0.076971,0.123819,0.171782,0.046691,0.169264,0.233215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,49.0,49.0,0.0,0.0,0.0,18.0,3.0,0.241202,0.088390,0.186091,0.227998,0.085253,0.139085,0.218705,0.062429,0.141967,0.244387
546,49.0,49.0,0.0,0.0,0.0,21.0,3.0,0.243181,0.091069,0.184918,0.227856,0.088603,0.140142,0.217205,0.062834,0.137809,0.240016
547,49.0,49.0,0.0,0.0,0.0,24.0,3.0,0.243590,0.093321,0.183162,0.229007,0.090276,0.139878,0.216148,0.063523,0.135354,0.237266
548,49.0,49.0,0.0,0.0,0.0,27.0,3.0,0.243374,0.094666,0.181884,0.230191,0.091255,0.139294,0.215286,0.064157,0.134192,0.235735


In [21]:
# Infer and score

for n_sp in log.attrs["n_species"]:
    for avg_samp_dt in log.attrs["avg_samp_dt"]:
        for meas_noise in log.attrs["meas_noise_list"]:
            datafiles = get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt)
            metadatafiles = get_files(datapath, n_sp, env_noise, meas_noise, avg_samp_dt, "metadata", "txt")

            for file_idx in range(len(datafiles)):
                datafile = datafiles[file_idx]
                metadatafile = metadatafiles[file_idx]
                metadict = get_meta(open(metadatafile, "r").read().split("\n"))
                
                df = pd.read_csv(datafile, index_col=0)
                
                param_columns = [f"r{i}" for i in range(1, n_sp+1)] + \
                [f"A{i},{j}" for i in range(1, n_sp+1) for j in range(1, n_sp+1)]
                cols = ["n_dset"] + list(df.columns[1:4]) + param_columns + ["MSPD", "CSR", "ES"]

                infer_out = pd.DataFrame(columns=cols)

                pd.options.mode.chained_assignment = None
                
                p = metadict["parameters"]
                r = p[:n_sp]
                A = p[n_sp:].reshape((n_sp,n_sp))

                # for i in tqdm(range(len(df.dataset.unique()))):
                for i in tqdm(range(30)):
                    if n_comb(len(df.dataset.unique()), i+1) < 10000:
                        combs = list(combinations(df.dataset.unique(), i+1))
                        np.random.shuffle(combs)
                        combs = combs[:100]
                    else:
                        combs = []
                        while len(combs) < 100:
                            comb = tuple(np.random.choice(df.dataset.unique(), i+1, replace=False))
                            if comb not in combs:
                                combs.append(comb)
                    for comb in combs:
                        comb = np.random.choice(df.dataset.unique(), i+1, replace=False)
                        df_comb = df[df.dataset.isin(comb)]
                        r_est, A_est = fit_ridge_cv(df_comb)
                        # r_est, A_est = fit_lasso_cv(df_comb)
                        # r_est, A_est = fit_elasticnet_cv(df_comb)
                        p_est = np.concatenate((r_est, A_est.flatten()))
                        MSPD = ((p-p_est)**2).mean()
                        CSR = (np.sign(A_est)==np.sign(A)).mean()
                        ES = calculate_es_score(A, A_est)
                        infer_out.loc[len(infer_out)] = [i+1, comb, avg_samp_dt, meas_noise] + list(p_est) + [MSPD, CSR, ES]

                infer_out.to_csv(datafile.split('dataset')[0]+"/inference"+datafile.split("dataset")[1])
                # infer_out.to_csv(datafile.split('dataset')[0]+"/inference_lasso"+datafile.split("dataset")[1])
                # infer_out.to_csv(datafile.split('dataset')[0]+"/inference_elasticnet"+datafile.split("dataset")[1])

100%|██████████| 30/30 [03:44<00:00,  7.48s/it]
